# Lecture 10B — Notebook 10B.1: Auditory Scales + Mel Filterbank Design

**Purpose:** Build intuition for auditory frequency warping and visualize mel filterbanks on the FFT grid.


In [ ]:
import os, json, math, re
from pathlib import Path

import numpy as np
import scipy.signal as sig
import scipy.fft as fft
import matplotlib.pyplot as plt

# Optional audio playback
try:
    from IPython.display import Audio, display
    HAS_IPY_AUDIO = True
except Exception:
    HAS_IPY_AUDIO = False

# Optional recording
try:
    import sounddevice as sd
    HAS_SD = True
except Exception as e:
    HAS_SD = False
    print("sounddevice not available (recording disabled).", e)

# ---------- Project paths ----------
PROJECT_ROOT = Path.cwd() / "EE519_L10B_Project"
REC_DIR = PROJECT_ROOT / "recordings"
FIG_DIR = PROJECT_ROOT / "figures"
RES_DIR = PROJECT_ROOT / "results"
MANIFEST_PATH = PROJECT_ROOT / "manifest.json"

for d in [REC_DIR, FIG_DIR, RES_DIR]:
    d.mkdir(parents=True, exist_ok=True)

def load_manifest(path=MANIFEST_PATH):
    if path.exists():
        return json.loads(path.read_text())
    return {"course":"EE519","lecture":"10B","created_utc":None,"clips":[]}

def save_manifest(manifest, path=MANIFEST_PATH):
    if manifest.get("created_utc") is None:
        manifest["created_utc"] = str(np.datetime64("now"))
    path.write_text(json.dumps(manifest, indent=2))
    print("Saved manifest:", path)

def save_fig(fig, name, dpi=150):
    out = FIG_DIR / name
    fig.savefig(out, dpi=dpi, bbox_inches="tight")
    print("Saved figure:", out)
    return out

# ---------- WAV I/O ----------
import wave
def write_wav(path: Path, x: np.ndarray, fs: int):
    x = np.asarray(x, dtype=np.float32)
    x = np.clip(x, -1.0, 1.0)
    x_i16 = (x * 32767.0).astype(np.int16)
    with wave.open(str(path), "wb") as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(fs)
        wf.writeframes(x_i16.tobytes())

def read_wav(path: Path):
    with wave.open(str(path), "rb") as wf:
        fs = wf.getframerate()
        n = wf.getnframes()
        x = np.frombuffer(wf.readframes(n), dtype=np.int16).astype(np.float32) / 32768.0
    return fs, x

def peak_normalize(x, target=0.98):
    m = np.max(np.abs(x)) + 1e-12
    return (x / m) * target

def play_audio(x, fs, label="audio"):
    if not HAS_IPY_AUDIO:
        print("(Audio playback not available)", label)
        return
    display(Audio(x, rate=fs))

def record_clip(seconds=2.0, fs=16000):
    if not HAS_SD:
        raise RuntimeError("sounddevice not available. Load wav files instead.")
    print(f"Recording {seconds:.1f}s @ {fs} Hz ...")
    x = sd.rec(int(seconds*fs), samplerate=fs, channels=1, dtype="float32")
    sd.wait()
    return fs, x.squeeze()

def add_clip_to_manifest(filename, label, fs, notes=""):
    clip = {
        "filename": filename,
        "label": label,
        "fs": int(fs),
        "notes": notes,
        "added_utc": str(np.datetime64("now")),
        "selections": {}
    }
    manifest["clips"].append(clip)
    save_manifest(manifest)
    return len(manifest["clips"]) - 1

def list_clips():
    for i,c in enumerate(manifest["clips"]):
        print(f"[{i}] {c['label']:14s}  {c['filename']}  fs={c['fs']}  notes={c.get('notes','')}")

# ---------- Selection + framing helpers ----------
def seconds_to_samples(t0, t1, fs, xlen):
    s0 = int(max(0, round(t0*fs)))
    s1 = int(min(xlen, round(t1*fs)))
    if s1 <= s0:
        raise ValueError("Bad selection: t1 must be > t0")
    return s0, s1

def samples_to_frame_range(s0, s1, N, H, xlen):
    f0 = max(0, int((s0 - N)//H) + 1)
    f1 = min(int((s1)//H), int((xlen-N)//H))
    if f1 < f0:
        f0 = max(0, int(s0//H))
        f1 = min(int((xlen-N)//H), f0)
    return f0, f1

def frame_signal(x, N, H):
    if len(x) < N:
        x = np.pad(x, (0, N-len(x)))
    num = 1 + (len(x) - N)//H
    idx = np.arange(N)[None,:] + H*np.arange(num)[:,None]
    return x[idx]

def db(x):
    return 20*np.log10(np.maximum(x, 1e-12))

# ---------- Auditory scale helpers ----------
def hz_to_mel(hz):
    return 2595.0 * np.log10(1.0 + hz/700.0)

def mel_to_hz(mel):
    return 700.0 * (10**(mel/2595.0) - 1.0)

def mel_filterbank(fs, nfft, n_mels=26, fmin=0.0, fmax=None):
    if fmax is None:
        fmax = fs/2
    # mel points
    mmin, mmax = hz_to_mel(fmin), hz_to_mel(fmax)
    m_pts = np.linspace(mmin, mmax, n_mels+2)
    hz_pts = mel_to_hz(m_pts)
    # fft bin frequencies
    freqs = np.linspace(0, fs/2, nfft//2 + 1)
    bins = np.floor((nfft+1) * hz_pts / fs).astype(int)
    fb = np.zeros((n_mels, len(freqs)), dtype=np.float64)
    for i in range(n_mels):
        b0, b1, b2 = bins[i], bins[i+1], bins[i+2]
        b0 = np.clip(b0, 0, len(freqs)-1)
        b1 = np.clip(b1, 0, len(freqs)-1)
        b2 = np.clip(b2, 0, len(freqs)-1)
        if b1 == b0: b1 += 1
        if b2 == b1: b2 += 1
        # rising
        fb[i, b0:b1] = (np.arange(b0, b1) - b0) / (b1 - b0 + 1e-12)
        # falling
        fb[i, b1:b2] = (b2 - np.arange(b1, b2)) / (b2 - b1 + 1e-12)
    return fb, freqs, hz_pts

manifest = load_manifest()
print("Project root:", PROJECT_ROOT)
print("Clips in manifest:", len(manifest["clips"]))


## 1) Hz ↔ Mel mapping (intuition)

We plot:
- mel as a function of Hz
- Hz as a function of mel

**Look for:** high resolution at low frequencies, compressed resolution at high frequencies.


In [ ]:
hz = np.linspace(0, 8000, 1000)
mel = hz_to_mel(hz)

fig = plt.figure(figsize=(7,3))
plt.plot(hz, mel)
plt.title("Mel scale warping: mel(hz)")
plt.xlabel("Frequency (Hz)"); plt.ylabel("Mel")
plt.tight_layout(); plt.show()
save_fig(fig, "L10B_mel_warping_mel_of_hz.png")

mel2 = np.linspace(0, hz_to_mel(8000), 1000)
hz2 = mel_to_hz(mel2)

fig = plt.figure(figsize=(7,3))
plt.plot(mel2, hz2)
plt.title("Inverse mel mapping: hz(mel)")
plt.xlabel("Mel"); plt.ylabel("Hz")
plt.tight_layout(); plt.show()
save_fig(fig, "L10B_mel_warping_hz_of_mel.png")


## 2) Build and plot mel filterbanks

We create triangular filters over FFT bins. Then visualize how they tile the spectrum.


In [ ]:
fs = 16000
NFFT = 1024
for n_mels in [10, 20, 40]:
    fb, freqs, hz_pts = mel_filterbank(fs, NFFT, n_mels=n_mels, fmin=0, fmax=fs/2)
    fig = plt.figure(figsize=(9,3))
    for i in range(fb.shape[0]):
        plt.plot(freqs, fb[i], linewidth=1.0)
    plt.title(f"Mel filterbank (n_mels={n_mels}) on FFT grid (NFFT={NFFT})")
    plt.xlabel("Frequency (Hz)"); plt.ylabel("Weight")
    plt.tight_layout(); plt.show()
    save_fig(fig, f"L10B_mel_filterbank_n{n_mels}_NFFT{NFFT}.png")


## 3) Micro-checkpoint: does tiling look right?

- Filters should overlap smoothly
- Low-frequency filters are narrower (more resolution)
- High-frequency filters are wider (less resolution)

If you see “flat” filters or all zeros:
- check fs and NFFT
- check fmin/fmax


## 4) Apply mel filterbank to a single frame

We take a clip + selection from manifest, pick a representative frame, and compute:
- FFT power spectrum
- Mel energies (filterbank dot power)


In [ ]:
list_clips()
CLIP_IDX = 0
segment_name = "seg1"

clip = manifest["clips"][CLIP_IDX]
fs, x = read_wav(REC_DIR/clip["filename"])
x = peak_normalize(x)

sel = clip["selections"]["analysis_segments"][segment_name]
N = sel["N"]; H = sel["H"]
f0, f1 = sel["frame_range"]
FRAME_IDX = int(round((f0+f1)/2))

frames = frame_signal(x, N, H)
window = sig.windows.hann(N, sym=False)
frame = frames[FRAME_IDX]*window

NFFT = 2048 if fs <= 16000 else 4096
X = fft.rfft(frame, n=NFFT)
pow_spec = (np.abs(X)**2) / NFFT
freqs = np.linspace(0, fs/2, len(pow_spec))

n_mels = 26
fb, freqs_fb, _ = mel_filterbank(fs, NFFT, n_mels=n_mels)
melE = fb @ pow_spec  # (n_mels,)

print("Frame:", FRAME_IDX, "melE shape:", melE.shape, "min/max:", melE.min(), melE.max())


In [ ]:
fig = plt.figure(figsize=(10,3))
plt.plot(freqs, db(pow_spec), linewidth=1.0)
plt.title("Power spectrum (dB) of representative frame")
plt.xlabel("Frequency (Hz)"); plt.ylabel("Power (dB)")
plt.tight_layout(); plt.show()
save_fig(fig, f"L10B_pow_spec_clip{CLIP_IDX}_{segment_name}_frame{FRAME_IDX}.png")

fig = plt.figure(figsize=(8,3))
plt.stem(np.arange(n_mels), np.log(melE + 1e-12), basefmt=" ")
plt.title("Log mel energies (single frame)")
plt.xlabel("Mel band index"); plt.ylabel("log energy")
plt.tight_layout(); plt.show()
save_fig(fig, f"L10B_log_melE_clip{CLIP_IDX}_{segment_name}_frame{FRAME_IDX}.png")


## Wrap-up
**What you learned:** mel warping + filterbank tiling + mel energies from a spectrum.  
**What’s next:** 10B.2 computes full log-mel spectrogram and compares to linear-frequency spectrogram.


## Reflection
1) Why are low-frequency mel filters narrower?  
2) How does number of mel bands trade off resolution vs stability?  
3) Where do you expect formants to show up in mel energies?
